In [102]:
import re
import os
import json
import pandas as pd

In [103]:
toc_file = "toc.txt"
chapter_data = "booktext.txt"

In [104]:
toc_array = []
file = open(toc_file)
for line in file:
    line = line.strip()
    if line != "\n" and re.search('[a-zA-Z0-9]', line):
        toc_array.append(line)
file.close()

In [105]:
book_array = []
file = open(chapter_data)
for line in file:
    line = line.strip()
    if line != "\n" and re.search('[a-zA-Z0-9]', line):
        book_array.append(line)
file.close()

In [106]:
def data_separation(line):
    line = line.strip()
    pos = line.find(" ")
    index_num = line[:pos]
    title = line[pos+1:]
    return (index_num, title)

def create_file(line):
    line = line.replace(" ", "_")
    file_name = "book_data/" + line + ".txt"
    file = open(file_name, "w+")
    file.close()
    return file_name

def match_line(line1, line2):
    line1 = line1.lower()
    line2 = line2.lower()
    index1, title = data_separation(line1)
    if index1 in line2 or index1+"." in line2:
        if title in line2: return 1
        else: return 0
    else:
        return 0

def clean_content(content):
    content = content.replace("\n", ". ")
    return content


In [109]:
toc_len = len(toc_array)
book_len = len(book_array)

current_line = toc_array[i]
next_line = toc_array[i + 1]

file_name= create_file(current_line)
current_file = open(file_name, "a")

for j in range(book_len):
    line = book_array[j]
    if match_line(next_line, line):
        if i < toc_len - 2:
            i += 1
            current_line = toc_array[i]
            next_line = toc_array[i + 1]
        else:
            current_line = toc_array[toc_len - 1]
        current_file.close()
        file_name = create_file(current_line)
        current_file = open(file_name, "a")
    else:
        line += "\n"
        current_file.write(line)
        
current_file.close()


In [113]:
book_data = {}
i = 0

for topic in toc_array:
    index_num, title = data_separation(topic)
    topic = topic.replace(" ", "_")
    file_name = "book_data/" + topic + ".txt"
    file = open(file_name, "r")
    content = file.read()
    content = clean_content(content)
    file.close()
    data = {
        'Section': index_num,
        'Title': title,
        'Content': content
    }
    book_data[i] = data
    i += 1

## Saving data in JSON Format

In [115]:
with open('book_data.json', 'w') as file:
    json.dump(book_data, file)

## Saving data in CSV format

In [118]:
df = pd.DataFrame(columns=['Section', 'Title', 'Content'])

for i in range(len(book_data)):
    df = df.append(book_data[i], ignore_index=True)

df.to_csv("book_data.csv")